<a href="https://colab.research.google.com/github/aso1901104/AI_teach2020/blob/master/2020AI0401_lang_character.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 異なる文字を使う言語を分類する

In [14]:
# ライブラリのimport
import numpy as np
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score

### 文字コードの値をインデックスにした配列を作り、各コード値(コードポイント)別に文章中の出現頻度をカウントする関数を定義


In [15]:
# Unicode文字列の文字コード値(コードポイント)頻度測定する関数
def count_codePoint(str):
  # Unicodeのコードポイントをインデックス位置(アドレス)とする配列を用意
  counter = np.zeros(65535)
  # 引数strの文字数だけループ雨
  for i in range( len(str) ):
    #各文字コード値を変数に代入
    # ord → 文字コード取得
    code_point  = ord(str[i])
    # Unicodeの範囲外だったら無視する
    if code_point > 65535:
      continue
    # endif
    # 対応するアドレスの出現回数カウントをインクリメント
    counter[code_point] += 1
  # endfor
  # 各要素を文字数で割って正規化(出現率、つまり％に変える)
  counter = counter / len(str)
  return counter

### 学習用データを作成する

In [16]:
# 学習用データの準備
ja_str = "これは日本語の文章です。"
en_str = "This is English Sentences."
th_str = "นี่เป็นประโยคภาษาไทย"
# それぞれの週つ限頻度割合を3つの要素として説明変数xを作る
x_train = [count_codePoint(ja_str), count_codePoint(en_str), count_codePoint(th_str)]
# 同じ順番で正解となる目的変数(ラベル)を作る
y_train = ['ja', 'en', 'th']


### 言語ごとの文字のパターンを学習する

In [17]:
# 今回はガウス分布によるナイーブベイズアルゴリズムで学習モデルを作る
# インスタンス取得
clf = GaussianNB()
# 説明変数と目的変数を渡す
clf.fit(x_train, y_train)

GaussianNB(priors=None, var_smoothing=1e-09)

### テスト検証用データを作成する

In [18]:
# テスト検証用データ(評価データ)の作成
ja_test_str = "こんにちは"
en_test_str = "Hello"
th_test_str = "สวัสดี"

# 説明変数にする
x_test = [count_codePoint(ja_test_str), count_codePoint(en_test_str), count_codePoint(th_test_str)]
#目的変数にする
y_test = ['ja', 'en', 'th']

### テスト検証(推論)し、評価する

In [19]:
# 推論させる
y_pred = clf.predict(x_test)
print("1つめの文章の言語は", y_pred[0],"です")
print("2つめの文章の言語は", y_pred[1],"です")
print("3つめの文章の言語は", y_pred[2],"です")
print("正解率：", accuracy_score(y_test, y_pred))

1つめの文章の言語は ja です
2つめの文章の言語は en です
3つめの文章の言語は th です
正解率： 1.0


## 同じ文字を使用する言語を分類

### サンプルとして使うWikipedia記事のテキストデータをダウンロードする

In [20]:
import urllib.request as req
# 学習データ用記事
url = "https://github.com/masatokg/sample_photo/raw/master/train.zip"
save_file = "train.zip"
req.urlretrieve( url, save_file )
# テスト検証用データ記事
url = "https://github.com/masatokg/sample_photo/raw/master/test.zip"
save_file = "test.zip"
req.urlretrieve( url, save_file )

# unzip(解凍する)
!unzip -o -q "./train.zip"
print("train.zip解凍")
!unzip -o -q "./test.zip"
print("test.zip解凍")

train.zip解凍
test.zip解凍


### ダウンロードしたテキストファイルのうち、学習用を読込み、学習用の説明変数データと目的変数データを作る

In [21]:
import glob

# 学習データの準備
index = 0
x_train = [] # 学習用説明変数
y_train = [] # 学習用目的変数（ラベル）
for file in glob.glob( "./train/*.txt" ): # train フォルダのtxt拡張子のファイル分ループ
  # 言語情報のキーワードになるものを文中から取得し、目的変数（ラベル）として設定
  y_train.append( file[8:10] )
  print("ファイルパスの一部をラベルにする:", file, " = ", file[8:10] )
  # print( y_train )

  # ファイル内の文字列を連結後、コードポイント頻度を測定し、説明変数として設定
  file_str = ""
  # ファイルを開いて1行ずつ文字列連結する
  for line in open( file, 'r' ):
    file_str = ( file_str + line )
  # for文終わり
  x_train.append( count_codePoint( file_str ) )
#for文終わり
# print( x_train )

ファイルパスの一部をラベルにする: ./train/de_cat.txt  =  de
ファイルパスの一部をラベルにする: ./train/en_cat.txt  =  en
ファイルパスの一部をラベルにする: ./train/es_dog.txt  =  es
ファイルパスの一部をラベルにする: ./train/es_elephant.txt  =  es
ファイルパスの一部をラベルにする: ./train/de_elephant.txt  =  de
ファイルパスの一部をラベルにする: ./train/de_dog.txt  =  de
ファイルパスの一部をラベルにする: ./train/en_elephant.txt  =  en
ファイルパスの一部をラベルにする: ./train/es_cat.txt  =  es
ファイルパスの一部をラベルにする: ./train/en_dog.txt  =  en


### 作成した学習データをガウス分布のナイーブベイズ分類器（GaussianNB）で学習させる

In [22]:
# 学習モデルのインスタンスを生成(ガウシアンNB)
clf = GaussianNB()
# 学習させる
clf.fit( x_train, y_train )

GaussianNB(priors=None, var_smoothing=1e-09)

### ダウンロードしたテキストファイルのうち、テスト検証用を読込、テスト検証用の説明変数、目的変数データを作る

In [23]:
# 評価（テスト検証）用データの作成
index = 0
x_test = [] # 説明変数
y_test = [] # 目的変数
# testフォルダ内のtxt拡張子のファイルをループする
for file in glob.glob( "./test/*.txt" ):
  # 言語情報になるキーワードをファイルパスから抜き出し、目的変数（ラベル）として設定
  y_test.append( file[7:9] )
  print("ファイルパスの一部をラベルにする:", file, " = ", file[7:9])

  # ファイル内の文字列を連結する
  file_str = ""
  # ファイル内の一行ずつを文字列連結
  for line in open( file, 'r' ):
    file_str = file_str + line
  # for文終わり
  # ファイル内の文字列のUnicode値（Unicodeポイント）の頻度を測定し、説明変数として設定
  x_test.append( count_codePoint( file_str ) )
  # print( file_str ) #1ファイル分を表示
  # print( x_test[0] )  #1ファイル分を表示
# for文終わり

ファイルパスの一部をラベルにする: ./test/en_lion.txt  =  en
ファイルパスの一部をラベルにする: ./test/es_lion.txt  =  es
ファイルパスの一部をラベルにする: ./test/de_lion.txt  =  de


### 学習モデルで推論し、評価する

In [24]:
# 推論する
y_pred = clf.predict( x_test )
print( y_pred ) # 推論結果を表示
# ひょうかする　
print( "正解率 = ", accuracy_score( y_test, y_pred ) )

['en' 'es' 'de']
正解率 =  1.0
